In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv', nrows=1_00_000)
test = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')

## Converting daytime object to datetime format

## It was just hard to do this I mean converting time like this

In [ ]:
train["DateTime"] = pd.to_datetime(train['pickup_datetime'], unit='ns')


In [ ]:
train['key'] = pd.to_datetime(train['key'], unit='ns')
train.info()

In [ ]:
test["DateTime"] = pd.to_datetime(test['pickup_datetime'], unit='ns')


In [ ]:
test["key"] = pd.to_datetime(test['key'], unit='ns')
test

**dropping null values**

In [ ]:
train.dropna(inplace=True)
test.dropna(inplace = True)

In [ ]:
train.shape

In [ ]:
# train = train.drop(['key','pickup_datetime'], axis = 1)
train = train.drop(['pickup_datetime'], axis = 1)
train.head()

In [ ]:
# test = test.drop(['key','pickup_datetime'], axis = 1)
test = test.drop(['pickup_datetime'], axis = 1)
test.head()

## Defining the labels and features

In [ ]:
y = train.iloc[0:,[0,1]]
x = train.iloc[0:,[2,3,4,5,6,7]]

**Adding distance to x**

In [ ]:
R = 6373.0
lat1 =np.asarray(np.radians(x['pickup_latitude']))
lon1 = np.asarray(np.radians(x['pickup_longitude']))
lat2 = np.asarray(np.radians(x['dropoff_latitude']))
lon2 = np.asarray(np.radians(x['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
ls1=[] 
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/ 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
distance = R * c

    
x['Distance']=np.asarray(distance)*0.621

# x_clean = x.drop([ 'pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis = 1)
# x_clean.head()
x.head()



**Adding distance to Train**

In [ ]:
R = 6373.0
lat1 =np.asarray(np.radians(train['pickup_latitude']))
lon1 = np.asarray(np.radians(train['pickup_longitude']))
lat2 = np.asarray(np.radians(train['dropoff_latitude']))
lon2 = np.asarray(np.radians(train['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
ls1=[] 
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/ 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
distance = R * c

    
train['Distance']=np.asarray(distance)*0.621

# x_clean = x.drop([ 'pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis = 1)
# x_clean.head()
train.head()


**Adding distance to test**

In [ ]:
R = 6373.0
lat11 =np.asarray(np.radians(test['pickup_latitude']))
lon11 = np.asarray(np.radians(test['pickup_longitude']))
lat22 = np.asarray(np.radians(test['dropoff_latitude']))
lon22 = np.asarray(np.radians(test['dropoff_longitude']))

dlon1 = lon22 - lon11
dlat1 = lat22 - lat11
ls1=[] 
a1 = np.sin(dlat1/2)**2 + np.cos(lat11) * np.cos(lat22) * np.sin(dlon1/ 2)**2
c1 = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1 - a1))
distance1 = R * c1

    
test['Distance']=np.asarray(distance1)*0.621
# test_clean = test.drop([ 'pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis = 1)

# test_clean.dtypes

test.head()

In [ ]:
x.dtypes

In [ ]:
y.dtypes

In [ ]:
print(x.head(),y.head(),test.head(),train.head())

## Training Starts


## Plotting graph of cost vs distance

In [ ]:
import matplotlib as plt
from sklearn import datasets,linear_model

In [ ]:
a = x.iloc[0:200,6]
b = y.iloc[0:200,1]


In [ ]:
import matplotlib.pyplot as plt


plt.scatter(b,a)
plt.show()



## Converting datetime to float()

In [ ]:
train['DateTime'] = pd.to_numeric(pd.to_datetime(train['DateTime']))
train['key'] = pd.to_numeric(pd.to_datetime(train['key']))
test['DateTime'] = pd.to_numeric(pd.to_datetime(test['DateTime']))
test['key'] = pd.to_numeric(pd.to_datetime(test['key']))
y['key'] = pd.to_numeric(pd.to_datetime(y['key']))

x['DateTime'] = pd.to_numeric(pd.to_datetime(x['DateTime']))


**Defining y for the prediction**

In [ ]:
y = y.iloc[:,1]

## Splitting the values

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

## Traning the model

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression(normalize=True)
lr.fit(x_train,y_train)
y_predicted = lr.predict(x_test)
print(lr.score(x_test,y_test))
# 1st test  = -0.013287637014155473

***Testing the accuracy***

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_predicted)
# # 1st test  = 112.004853684073
## with 100000 dataset = 94.0358896381812
## with 1000000 dataset try 1 = 95.36892390730144
## with 1000000 dataset try 2 = 95.36026307535313
## with 1000000 dataset try 3 = 96.83297952994907
## with 100000 dataset  = 93.37144467436306
## with 10000 dataset try1 = 96.03901121564002
## with 10000 dataset try2 = 81.26929962886342
## with 10000 dataset try3 = 98.48885378200688
## with 10000 dataset try4 = 104.36002792791918
## with 10000 dataset try5 = 97.20492555017101
## with 10000 dataset try6 = 83.93965348592229
## with 10000 dataset try7 = 75.27941712945345
## with 10000 dataset try8  = 78.95396330172005
## with 1000 dataset   =   39.90092494230721
## with 1000 dataset   =   76.44837215099844
## with 1000 dataset   =   38.46728504333877
## with 1000 dataset   =   93.07429700649337
## with 1000 dataset   =   48.39973592705823
## with 1000 dataset   =   1055865.4214096582
## with 1000 dataset   =   85.46861840805632
## with 1000 dataset   =   64.39358773994795
## with 100 dataset    =   12.34566678724564
## with 100 dataset    =   10.142841154191558 
## with 100 dataset    =   5.786482441190635
## with 100 dataset    =   30.963445495727807
## with 100 dataset    =   14.648523611686821


## Submission starts


In [ ]:
pred=np.round(lr.predict(test.drop('key',axis=1)),2)
pred.shape

**Reading the submission file for some insight**

In [ ]:
pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv').head()

In [ ]:

Submission=pd.DataFrame(data=pred,columns=['fare_amount'])
Submission['key']=test['key']
Submission=Submission[['key','fare_amount']]

## File Submitted

In [ ]:
Submission.set_index('key',inplace=True)